# Run custom versions of the FastScape landscape evolution model

Let's import some packages first (you can install them using ``conda``).

In [ ]:
import numpy as np
import xarray as xr
import xsimlab as xs
import fastscape

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Create a new component (process)

Sometimes there is no existing module for the simulation that we'd like to run. The ``xarray-simlab`` framework allows to create new components and plug them to existing models with very little effort.

Here is an example below that chooses randomly spatially variable values for the $K$ parameter of the stream-power law.

In [ ]:
from fastscape.processes import StreamPowerChannel, RasterGrid2D

A ``xarray-simlab`` component (or process) is a (most-often) small, "classic" Python class that is decorated by ``xsimlab.process``.

More info on how to create processes: https://xarray-simlab.readthedocs.io/en/latest/create_model.html

In [ ]:
@xs.process
class SPLRandomK:
    grid_shape = xs.foreign(RasterGrid2D, 'shape')
    k_coef = xs.foreign(StreamPowerChannel, 'k_coef', intent='out')
    
    def initialize(self):
        self.k_coef = np.random.uniform(1e-5, 1e-4, size=self.grid_shape)

## Update the basic model with this new component

Let's first import ``basic_model``

In [ ]:
from fastscape.models import basic_model

basic_model

Then we just need ``update_processes``: 

In [ ]:
new_model = basic_model.update_processes({'randk': SPLRandomK})

new_model

## Run the new model (reuse an existing setup)

Let's import below the setup that we have created in the ``run_basic_model`` notebook:

In [ ]:
in_ds = xr.load_dataset('basic_input.nc')

Let's update the setup and run the new model created above:

In [ ]:
with new_model:
    out_ds = (
        in_ds.xsimlab.filter_vars()
             .xsimlab.update_vars(output_vars={'randk__k_coef': None})
             .xsimlab.run()
    )


out_ds

Let's look at the topography:

In [ ]:
import hvplot.xarray
import matplotlib.pyplot as plt

out_ds.topography__elevation.hvplot.image(x='x', y='y',
                                          cmap='viridis',
                                          groupby='out')

## Exercices

- Update the component so that the user can provide a range for the randomly generated parameter values
- Update the component so that new random values are generated at each time step
- Any idea scientifically more interesting?